In [18]:
import pandas as pd
import geopandas as gpd

In [19]:
# GLOBAL VARIABLES

# seg summary fields to be useds
use_fields = ['SEGID','DY_VOL','FT','ATYPENAME','LANES']

# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
fnSegmentsShapefile = 'data/segments/Segments_State_20231219_Draft.shp'

# field for weekday factor
colFacWdAvg = 'FAC_WDAVG'
colFacSpr   = 'FAC_SPR'
colFacFal   = 'FAC_FAL'

# SHOULD WE ADJUST WEEKDAY FACTOR INTO FUTURE?? 

# base year adjustment
# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
sourceName = 'AADTHistory.xlsx'
baseYear = 2019

dfModelSegSummaries = pd.DataFrame( [
    [0, 2019, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/BY2019_Summary_SEGID.csv'      ],
    [0, 2023, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/OY2023_Summary_SEGID.csv'      ],
    [0, 2028, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/STIP2028_Summary_SEGID.csv'    ],
    [0, 2032, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2032_Summary_SEGID.csv'      ],
    [0, 2042, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2042_Summary_SEGID.csv'      ],
    [0, 2050, 'data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2050_Summary_SEGID.csv'      ],
    [1, 2019, 'data/model-output/1_WF/v9_SE19_Net19_Summary_SEGID_withnewSegIds.csv'           ],
    [1, 2023, 'data/model-output/1_WF/v9_SE23_Net23_Summary_SEGID_withnewSegIds.csv'           ],
    [1, 2028, 'data/model-output/1_WF/v9_TIP_SE28_Net28_Summary_SEGID_withnewSegIds.csv'       ],
    [1, 2032, 'data/model-output/1_WF/v9_RTP_SE32_Net32_Summary_SEGID_withnewSegIds.csv'       ],
    [1, 2042, 'data/model-output/1_WF/v9_RTP_SE42_Net42_Summary_SEGID_withnewSegIds.csv'       ],
    [1, 2050, 'data/model-output/1_WF/v9_RTP_SE50_Net50_Summary_SEGID_withnewSegIds.csv'       ],
    [2, 2019, 'data/model-output/2_Cache/BY2019_Summary_SEGID.dbf'                             ],
    [2, 2023, 'data/model-output/2_Cache/BY2023_Summary_SEGID.dbf'                             ],
    [2, 2028, 'data/model-output/2_Cache/STIP_2028_Summary_SEGID.dbf'                          ],
    [2, 2032, 'data/model-output/2_Cache/FY2032_Summary_SEGID.dbf'                             ],
    [2, 2042, 'data/model-output/2_Cache/FY2042_Summary_SEGID.dbf'                             ],
    [2, 2050, 'data/model-output/2_Cache/FY2050_Summary_SEGID.dbf'                             ],
    [3, 2019, 'data/model-output/3_Dixie/Summary_SEGID_19.dbf'                                 ],
    [3, 2023, 'data/model-output/3_Dixie/Summary_SEGID_23.dbf'                                 ],
    [3, 2028, 'data/model-output/3_Dixie/Summary_SEGID_28.dbf'                                 ],
    [3, 2032, 'data/model-output/3_Dixie/Summary_SEGID_32.dbf'                                 ],
    [3, 2042, 'data/model-output/3_Dixie/Summary_SEGID_42.dbf'                                 ],
    [3, 2050, 'data/model-output/3_Dixie/Summary_SEGID_50.dbf'                                 ],
    [4, 2019, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/BY2019_Summary_SEGID.dbf'           ],
    [4, 2023, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/OY2023_Summary_SEGID.dbf'           ],
    [4, 2028, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/STIP2028_Summary_SEGID.dbf'         ],
    [4, 2032, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/FY2032_Summary_SEGID.dbf'           ],
    [4, 2042, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/FY2042_Summary_SEGID.dbf'           ],
    [4, 2050, 'data/model-output/4_SuWsv2_2023-09-13_DRAFT/FY2050_Summary_SEGID.dbf'           ],
    [5, 2019, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_BY2019.dbf'  ],
    [5, 2023, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_OY2023.dbf'  ],
    [5, 2028, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_STIP2028.dbf'],
    [5, 2032, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2032.dbf'  ],
    [5, 2042, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2042.dbf'  ],
    [5, 2050, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2050.dbf'  ]
], columns=('modSubareaId','modYear','modSegSummaryFile'))

#display(dfModelSegSummaries)


In [20]:
# Required libraries
from dbfread import DBF
import pandas as pd

# Create a list to store DataFrames read from each file
frames = []

# Iterate through the rows and read each file
for index, row in dfModelSegSummaries.iterrows():
    # Prepend path with 'dbfs:/'
    file_path = row['modSegSummaryFile']
    print(file_path)
    
    # Check the file extension
    if file_path.endswith('.dbf'):
        # Read the .dbf file with dbfread
        table = DBF(file_path)
        df = pd.DataFrame(iter(table))  # Convert the DBF table to a DataFrame
    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    else:
        # If the file is neither .csv nor .dbf, skip this iteration
        print(f"Unsupported file format for file: {file_path}")
        continue

    df = df[use_fields]
    df['YEAR'     ] = row['modYear'     ] # Add modYear column
    df['SUBAREAID'] = row['modSubareaId'] # Add modSubareaId column
    frames.append(df)

# Concatenate all the frames into a single DataFrame
dfMdlVol = pd.concat(frames, ignore_index=True)

# If you're using Databricks, use 'display' to show the DataFrame
display(dfMdlVol)

data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/BY2019_Summary_SEGID.csv


C:\Users\bhereth\AppData\Local\Temp\ipykernel_1408\1412907378.py:20: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\bhereth\AppData\Local\Temp\ipykernel_1408\1412907378.py:20: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/OY2023_Summary_SEGID.csv
data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/STIP2028_Summary_SEGID.csv


C:\Users\bhereth\AppData\Local\Temp\ipykernel_1408\1412907378.py:20: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\bhereth\AppData\Local\Temp\ipykernel_1408\1412907378.py:20: DtypeWarning: Columns (153) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2032_Summary_SEGID.csv
data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2042_Summary_SEGID.csv
data/model-output/0_USTM_v3.0 - 2023-08-17_DRAFT/FY2050_Summary_SEGID.csv
data/model-output/1_WF/v9_SE19_Net19_Summary_SEGID_withnewSegIds.csv
data/model-output/1_WF/v9_SE23_Net23_Summary_SEGID_withnewSegIds.csv
data/model-output/1_WF/v9_TIP_SE28_Net28_Summary_SEGID_withnewSegIds.csv
data/model-output/1_WF/v9_RTP_SE32_Net32_Summary_SEGID_withnewSegIds.csv
data/model-output/1_WF/v9_RTP_SE42_Net42_Summary_SEGID_withnewSegIds.csv
data/model-output/1_WF/v9_RTP_SE50_Net50_Summary_SEGID_withnewSegIds.csv
data/model-output/2_Cache/BY2019_Summary_SEGID.dbf
data/model-output/2_Cache/BY2023_Summary_SEGID.dbf
data/model-output/2_Cache/STIP_2028_Summary_SEGID.dbf
data/model-output/2_Cache/FY2032_Summary_SEGID.dbf
data/model-output/2_Cache/FY2042_Summary_SEGID.dbf
data/model-output/2_Cache/FY2050_Summary_SEGID.dbf
data/model-output/3_Dixie/Summary_SEGID

,SEGID,DY_VOL,FT,ATYPENAME,LANES,YEAR,SUBAREAID
0,Cache_4013,659.7,6.0,Transition,1.0,2019,0
1,0006_000.0,126.7,2.0,Rural,1.0,2019,0
2,0006_000.7,213.3,2.0,Rural,1.0,2019,0
3,0006_016.0,76.6,2.0,Rural,1.0,2019,0
4,0006_046.0,76.6,2.0,Rural,1.0,2019,0
...,...,...,...,...,...,...,...
78717,NO,510.8,6.0,Rural,2.0,2050,5
78718,UDOT_7006,4181.0,4.0,Transition,4.0,2050,5
78719,UDOT_7007,7259.0,4.0,Rural,4.0,2050,5
78720,UDOT_7008,453.9,4.0,Rural,2.0,2050,5


In [21]:
dfMdlVol[dfMdlVol['YEAR']==0]

,SEGID,DY_VOL,FT,ATYPENAME,LANES,YEAR,SUBAREAID


In [22]:
# GET WEEKDAY FACTORS

# read in segment shapefile
gdfSegments = gpd.read_file(fnSegmentsShapefile)
dfFac = gdfSegments[['SEGID', 'SUBAREAID', 'CO_FIPS', colFacWdAvg, colFacSpr, colFacFal]].copy()

dfFac['FAC_SPRFAL'] = (dfFac[colFacSpr] + dfFac[colFacFal]) / 2

display(dfFac)

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL
0,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960
1,0006_000.7,0,27,0.9840,1.0276,1.0316,1.02960
2,0006_016.0,0,27,0.9840,1.0276,1.0316,1.02960
3,0006_046.0,0,27,0.9840,1.0276,1.0316,1.02960
4,0006_060.2,0,27,0.9840,1.0276,1.0316,1.02960
...,...,...,...,...,...,...,...
8923,WFRC_8429,1,57,1.0918,1.0107,1.0324,1.02155
8924,WFRC_8430,1,35,1.0946,1.0117,1.0095,1.01060
8925,WFRC_8431,1,57,1.0918,1.0107,1.0324,1.02155
8926,WFRC_8432,1,57,0.9840,1.0276,1.0316,1.02960


In [23]:
dfFac.loc[(dfFac['FAC_WDAVG']==0), 'checkZero'] = 1
dfFac.groupby(['SUBAREAID'],as_index = False).agg(numSegs=('SEGID','count'),numSegsWithFacZero=('checkZero','sum'))

,SUBAREAID,numSegs,numSegsWithFacZero
0,0,2576,0.0
1,1,4620,78.0
2,2,489,0.0
3,3,615,0.0
4,4,379,0.0
5,5,249,0.0


In [24]:
import numpy as np

# Your existing code for merging and filling NA
_df = pd.DataFrame.merge(dfFac, dfMdlVol, on=('SEGID','SUBAREAID'), how='inner')
_df.fillna(0, inplace=True)
display()

# calculate AADT by subareaid
# divide by colWeekdayFactor
_df.loc[(~(_df['SUBAREAID'].isin([4])) & (_df[colFacWdAvg]> 0)), 'modAadt'] = (_df['DY_VOL'] / _df[colFacWdAvg])
_df.loc[(~(_df['SUBAREAID'].isin([4])) & (_df[colFacWdAvg]==0)), 'modAadt'] = 0

# divide by average of Spring and Fall only for Summit/Wasatch Model
_df.loc[(_df['SUBAREAID'].isin([4]) & (_df['FAC_SPRFAL']>0 )), 'modAadt'] = (_df['DY_VOL'] / _df[colFacWdAvg] / _df['FAC_SPRFAL'])
_df.loc[(_df['SUBAREAID'].isin([4]) & (_df['FAC_SPRFAL']==0)), 'modAadt'] = 0

# Convert to integer as you had before
_df['modAadt'] = _df['modAadt'].round(0).astype(int)

dfModAadt = _df

# Display the DataFrame
display(dfModAadt)

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt
0,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,126.7,2.0,Rural,1.0,2019,129
1,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,99.7,2.0,Rural,1.0,2023,101
2,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,101.7,2.0,Rural,1.0,2028,103
3,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,103.6,2.0,Rural,1.0,2032,105
4,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,107.9,2.0,Rural,1.0,2042,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46414,WFRC_8431,1,57,1.0918,1.0107,1.0324,1.02155,0.0,631.1,4.0,Transition,2.0,2032,578
46415,WFRC_8431,1,57,1.0918,1.0107,1.0324,1.02155,0.0,1676.3,4.0,Transition,2.0,2042,1535
46416,WFRC_8431,1,57,1.0918,1.0107,1.0324,1.02155,0.0,7735.0,4.0,Transition,2.0,2050,7085
46417,WFRC_8432,1,57,0.9840,1.0276,1.0316,1.02960,0.0,22399.6,14.0,Transition,4.0,2050,22764


In [25]:
dfModAadt[dfModAadt['SUBAREAID'].isin([4,5]) & (dfModAadt['FAC_SPRFAL']>0)]

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt
1290,0014_000.0,5,21,1.0946,1.0117,1.0095,1.01060,0.0,7343.8,3.0,Urban,4.0,2019,6709
1291,0014_000.0,5,21,1.0946,1.0117,1.0095,1.01060,0.0,7789.2,3.0,Urban,4.0,2023,7116
1292,0014_000.0,5,21,1.0946,1.0117,1.0095,1.01060,0.0,8306.7,3.0,Urban,4.0,2028,7589
1293,0014_000.0,5,21,1.0946,1.0117,1.0095,1.01060,0.0,7797.2,3.0,Urban,4.0,2032,7123
1294,0014_000.0,5,21,1.0946,1.0117,1.0095,1.01060,0.0,8210.4,3.0,Urban,4.0,2042,7501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44966,UDOT_7043,5,21,1.0918,1.0107,1.0324,1.02155,0.0,0.0,4.0,Transition,2.0,2023,0
44967,UDOT_7043,5,21,1.0918,1.0107,1.0324,1.02155,0.0,0.0,4.0,Transition,2.0,2028,0
44968,UDOT_7043,5,21,1.0918,1.0107,1.0324,1.02155,0.0,0.0,4.0,Transition,2.0,2032,0
44969,UDOT_7043,5,21,1.0918,1.0107,1.0324,1.02155,0.0,0.0,4.0,Transition,2.0,2042,0


In [26]:
_df = pd.read_csv('intermediate/aadt.csv')
_df

,SEGID,YEAR,AADT,SOURCE
0,0006_000.0,1981,325,AADTHistory.xlsx
1,0006_000.0,1982,335,AADTHistory.xlsx
2,0006_000.0,1983,430,AADTHistory.xlsx
3,0006_000.0,1984,580,AADTHistory.xlsx
4,0006_000.0,1985,585,AADTHistory.xlsx
...,...,...,...,...
178826,3483_000.0,2017,990,AADTHistory.xlsx
178827,3483_000.0,2018,1030,AADTHistory.xlsx
178828,3483_000.0,2019,1042,AADTHistory.xlsx
178829,3483_000.0,2020,1041,AADTHistory.xlsx


In [27]:
# GET BASE YEAR OBSERVED

# read intermediate CSV with base year observed AADT
_df = pd.read_csv('intermediate/aadt.csv')

# filter by source and year
_df = _df[(_df['SOURCE']==sourceName) & (_df['YEAR']<=baseYear)]

# get max row of max year for each segid... incase there is no observed 2019 data, but maybe a earlier year
# Then, group by 'SEGID' and find the index of the maximum 'YEAR' for each group
idx = _df.groupby('SEGID')['YEAR'].idxmax()

# Use the index to filter the rows
_df = _df.loc[idx]

_df = _df[['SEGID','AADT']]
_df.rename(columns={'AADT':'obsAadt'}, inplace=True)

dfObsAadt_BaseYear = _df

display(dfObsAadt_BaseYear)

,SEGID,obsAadt
37,0006_000.0,415
77,0006_000.7,415
117,0006_016.0,415
157,0006_046.0,372
197,0006_060.2,372
...,...,...
178781,3468_000.0,2581
178793,3469_000.0,350
178798,3470_000.0,1724
178813,3478_000.0,911


In [28]:
# CALCULATE ADJUSTMENT FACTOR FROM BASE YEAR MODEL VS OBSERVED
dfModAadt_BaseYear = dfModAadt[dfModAadt['YEAR']==2019]

_df = pd.DataFrame.merge(dfModAadt_BaseYear[use_fields + ['modAadt']], dfObsAadt_BaseYear[['SEGID','obsAadt']], on='SEGID', how='left')
_df.fillna(0,inplace=True)

_df['aadtAdjFactor'] = _df['obsAadt'] - _df['modAadt']

_df.loc[(_df['obsAadt']==0), 'aadtAdjFactor'] = 0

display(_df[_df['modAadt']==0])

dfModAdjFactor = _df[['SEGID','aadtAdjFactor']]

display(dfModAdjFactor)


,SEGID,DY_VOL,FT,ATYPENAME,LANES,modAadt,obsAadt,aadtAdjFactor
435,0021_077.2,0.0,3.0,Transition,1.0,0,2605.0,2605.0
510,0025_000.0,0.0,4.0,Rural,1.0,0,179.0,179.0
511,0025_002.9,0.0,4.0,Rural,1.0,0,179.0,179.0
2010,0122_000.0,0.0,4.0,Rural,1.0,0,77.0,77.0
2066,0128_000.0,0.0,4.0,Rural,1.0,0,1055.0,1055.0
...,...,...,...,...,...,...,...,...
7428,WFRC_8195,0.0,4.0,Rural,2.0,0,0.0,0.0
7437,WFRC_8217,0.0,4.0,Suburban,2.0,0,0.0,0.0
7462,WFRC_8246,0.0,5.0,Suburban,2.0,0,0.0,0.0
7482,WFRC_8319,0.0,2.0,Suburban,4.0,0,0.0,0.0


,SEGID,aadtAdjFactor
0,0006_000.0,286.0
1,0006_000.7,198.0
2,0006_016.0,337.0
3,0006_046.0,294.0
4,0006_060.2,290.0
...,...,...
7524,WFRC_8421,0.0
7525,WFRC_8422,0.0
7526,WFRC_8423,0.0
7527,WFRC_8430,0.0


In [29]:
# Function to round based on the given ranges
def custom_rounding(value):
    if 0 <= value < 100:
        return round(value / 10) * 10
    elif 100 <= value < 1000:
        return round(value / 50) * 50
    elif 1000 <= value < 10000:
        return round(value / 100) * 100
    elif 10000 <= value < 100000:
        return round(value / 500) * 500
    elif value >= 100000:
        return round(value / 1000) * 1000
    else:
        return value

In [30]:
# ADJUST ALL MODEL VOLUMES BASED ON BASE YEAR ADJ FACTOR

dfModWithAdj = pd.DataFrame.merge(dfModAadt, dfModAdjFactor, on='SEGID', how='left')
dfModWithAdj['aadtAdjFactor'].fillna(0,inplace=True)
dfModWithAdj['modForecast'] = (dfModWithAdj['modAadt'] + dfModWithAdj['aadtAdjFactor']).apply(custom_rounding)
dfModWithAdj

# remove 2019 since it is base
#dfModWithAdjNo2019 = dfModWithAdj[dfModWithAdj['YEAR']>2019]

# sort
#dfModWithAdjNo2019 = dfModWithAdjNo2019.sort_values(by=['SEGID', 'YEAR'], ascending=[True, True])

#dfModWithAdjNo2019

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt,aadtAdjFactor,modForecast
0,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,126.7,2.0,Rural,1.0,2019,129,286.0,400.0
1,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,99.7,2.0,Rural,1.0,2023,101,286.0,400.0
2,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,101.7,2.0,Rural,1.0,2028,103,286.0,400.0
3,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,103.6,2.0,Rural,1.0,2032,105,286.0,400.0
4,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,107.9,2.0,Rural,1.0,2042,110,286.0,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46414,WFRC_8431,1,57,1.0918,1.0107,1.0324,1.02155,0.0,631.1,4.0,Transition,2.0,2032,578,0.0,600.0
46415,WFRC_8431,1,57,1.0918,1.0107,1.0324,1.02155,0.0,1676.3,4.0,Transition,2.0,2042,1535,0.0,1500.0
46416,WFRC_8431,1,57,1.0918,1.0107,1.0324,1.02155,0.0,7735.0,4.0,Transition,2.0,2050,7085,0.0,7100.0
46417,WFRC_8432,1,57,0.9840,1.0276,1.0316,1.02960,0.0,22399.6,14.0,Transition,4.0,2050,22764,0.0,23000.0


In [31]:
# check if base year adjusted equals base year observed

_df1 = dfModWithAdj[dfModWithAdj['YEAR']==baseYear]
_df2 = dfObsAadt_BaseYear

_df3 = pd.DataFrame.merge(_df1, _df2, on='SEGID')

_df3['ModVsObs'] = _df3['modForecast'] - _df3['obsAadt']

display(_df3[_df3['ModVsObs']!=0])


,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt,aadtAdjFactor,modForecast,obsAadt,ModVsObs
0,0006_000.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,126.7,2.0,Rural,1.0,2019,129,286.0,400.0,415,-15.0
1,0006_000.7,0,27,0.9840,1.0276,1.0316,1.02960,0.0,213.3,2.0,Rural,1.0,2019,217,198.0,400.0,415,-15.0
2,0006_016.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,76.6,2.0,Rural,1.0,2019,78,337.0,400.0,415,-15.0
3,0006_046.0,0,27,0.9840,1.0276,1.0316,1.02960,0.0,76.6,2.0,Rural,1.0,2019,78,294.0,350.0,372,-22.0
4,0006_060.2,0,27,0.9840,1.0276,1.0316,1.02960,0.0,80.4,2.0,Rural,1.0,2019,82,290.0,350.0,372,-22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6703,3466_000.3,1,57,1.0924,1.0104,1.0243,1.01735,0.0,2175.1,4.0,Suburban,2.0,2019,1991,5489.0,7500.0,7480,20.0
6704,3468_000.0,0,57,0.9593,0.9859,1.0030,0.99445,0.0,901.9,4.0,Rural,1.0,2019,940,1641.0,2600.0,2581,19.0
6706,3470_000.0,1,57,1.0924,1.0104,1.0243,1.01735,0.0,6090.0,5.0,Suburban,2.0,2019,5575,-3851.0,1700.0,1724,-24.0
6707,3478_000.0,0,57,0.9555,0.8719,0.9353,0.90360,0.0,469.7,4.0,Transition,1.0,2019,492,419.0,900.0,911,-11.0


In [32]:
# export final file
dfModWithAdj.to_csv('intermediate/model-forecasts.csv', index=False)

In [33]:
dfModWithAdj[dfModWithAdj['SEGID']=='0085_011.9']

,SEGID,SUBAREAID,CO_FIPS,FAC_WDAVG,FAC_SPR,FAC_FAL,FAC_SPRFAL,checkZero,DY_VOL,FT,ATYPENAME,LANES,YEAR,modAadt,aadtAdjFactor,modForecast
7890,0085_011.9,1,35,1.0918,1.0107,1.0324,1.02155,0.0,63100.4,13.0,Suburban,4.0,2028,57795,0.0,58000.0
7891,0085_011.9,1,35,1.0918,1.0107,1.0324,1.02155,0.0,98684.8,34.0,Suburban,8.0,2032,90387,0.0,90500.0
7892,0085_011.9,1,35,1.0918,1.0107,1.0324,1.02155,0.0,125799.8,34.0,Urban,8.0,2042,115222,0.0,115000.0
7893,0085_011.9,1,35,1.0918,1.0107,1.0324,1.02155,0.0,155227.8,34.0,Urban,12.0,2050,142176,0.0,142000.0
